In [1]:
import pandas as pd
import sys
from io import StringIO
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets

%matplotlib inline
tab = widgets.Tab()    
out = widgets.Output(layout={'border': '1px solid black'})
up = widgets.FileUpload(accept=".csv", multiple=False)
delim = widgets.RadioButtons(
    options=[';', ',', ' '],
    description='Separator: ',
    disabled=False)
eraser = widgets.SelectMultiple(
    options=['tab','"'],
    value=['tab'],
    #rows=10,
    description='Eraser: ',
    disabled=False)
rows = widgets.IntSlider(
    value=0,
    step=1,
    description='# of lines:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d')
button_upload = widgets.Button(
    description='Upload',
    disabled=False,
    button_style='warning',
    tooltip='Click to Upload',
    icon='check')
button_preview = widgets.Button(
    description='Preview',
    disabled=False,
    button_style='info',
    tooltip='Click to Preview',
    icon='search')
button_plot = widgets.Button(
    description='Plot',
    disabled=False,
    button_style='danger',
    tooltip='Click to Plot',
    icon='pencil')

toggle = widgets.ToggleButtons(
    options=['Preview  ', 'Info  ', 'Stats  '],
    description='Options',
    disabled=False,
    button_style='warning',
    icons=['search', 'info', 'tachometer'])
accordion = widgets.Accordion(children=[
    up, 
    widgets.VBox([delim, eraser]), 
    rows])
accordion.set_title(0, 'File Selection')
accordion.set_title(1, 'Preprocessing')
accordion.set_title(2, 'Skip Rows')
accordion_box = widgets.VBox([
    accordion, 
    widgets.HBox([button_preview, button_upload]),
    out
])

In [2]:
children = [
    accordion_box, 
    widgets.VBox([toggle, out]),
    widgets.VBox([
        button_plot,
        out
])]
tab.children = children
tab.set_title(0, "Upload")
tab.set_title(1, "Describer")
tab.set_title(2, "Plotter")

In [12]:
def content_parser():
    if up.value == {}:
        with out:
            print('No CSV loaded')    
    else:
        typ, content = "", ""
        up_value = up.value
        for i in up_value.keys():
            typ = up_value[i]['metadata']["type"]
            if typ == "csv":
                content = up_value[i]["content"]
                content_str = str(content, 'utf-8')
    
            if eraser.value != {}: 
                for val in eraser.value:
                    if val == "tab":
                        content_str = content_str.lower()
                    else:
                        content_str = content_str.replace(val,"")
            if content_str != "":
                str_io = StringIO(content_str) 
                return str_io

In [13]:
def remove(tweet):
    
    #case folding tweet
    tweet = tweet.lower()
    
    #remove one char
    tweet = re.sub(r"\b[a-zA-Z]\b", "", tweet)

    #remove angka
    tweet = re.sub('[0-9]+', '', tweet)
    
    # remove tab, new line, ans back slice
    tweet = tweet.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
 
    # remove non ASCII (emoticon, chinese word, .etc)
    tweet = tweet.encode('ascii', 'replace').decode('ascii')
    
    # remove mention, link, hashtag
    tweet = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", tweet).split())

    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    
    #remove punctuation
    tweet = tweet.translate(str.maketrans("","",string.punctuation))
    
    #remove whitespace leading & trailing
    tweet = tweet.strip()
    
    #remove multiple whitespace into single whitespace
    tweet = re.sub('\s+',' ',tweet)

    return tweet

In [4]:
def df_converter():
    content = content_parser()
    if content is not None:
        df = pd.read_csv(content, sep=delim.value, index_col=False, skiprows=rows.value)    
        return df
    else:
        return None

In [5]:
def preview():
    df = df_converter()
    with out:
        out.clear_output()
        print('\n -----Now this is how your DF looks like:----- \n')
        if df is not None:
            print(df.head(10))
        else:
            print('Configuration is wrong/missing...')

In [6]:
def upload():
    df = df_converter()
    with out:
        out.clear_output()
        print('\n --------Your uploaded DF looks like:-------- \n')
        if df is not None:
            print(df)
            x_axis.options = df.columns
            y_axis.options = df.columns
        else:
            print('Configuration is wrong/missing...')

In [7]:
def desc():
    info_level = toggle.value
    if info_level != {}:
        df = df_converter()
        with out:
            out.clear_output()
            print('\n ------Your {} looks like:------ \n'.format(info_level))
            if df is not None:
                if info_level == 'Info  ':
                    print(df.info(verbose=True))
                elif info_level == 'Stats  ':
                    print(df.describe())
                elif info_level == 'Preview  ':
                    print(df.head(5))
                else:
                    print('Configuration is wrong/missing...')

In [8]:
def plot():
        values = [31, 6, 13]
        explode = [0, 0.1, 0]
        labels = ['Positif', 'Negatif', 'Netral']
        colors = ['brown', 'lightcoral', 'beige']
        
        def make_autopct(values):
            def my_autopct(pct):
                total  = sum(values)
                val = int(round(pct*total/100.0))
                return '{p:.2f}%({v:d})'.format(p=pct,v=val)
            return my_autopct
        
        plt.pie(values, colors=colors, labels=labels, explode=explode,
                autopct=make_autopct(values), shadow=True, startangle=90)
        plt.legend()
        plt.title("Hasil Feedback Pelayanan ShopeeFood")
        plt.show()

In [9]:
                
def preview_clicked(b):
    preview()
def upload_clicked(b):
    upload()
def desc_clicked(b):
    desc()
def plotter_clicked(b):
    plot()

In [10]:
button_preview.on_click(preview_clicked)
button_upload.on_click(upload_clicked)
toggle.observe(desc_clicked, 'value')
button_plot.on_click(plotter_clicked)

In [11]:
display(tab)

UnboundLocalError: local variable 'content_str' referenced before assignment